In [ ]:
import os
from itertools import product

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
from itertools import product
import os
import pylab
from statsmodels import robust
from itertools import product


import math
from matplotlib.transforms import offset_copy
from matplotlib import gridspec

import scipy
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

import pickle

In [ ]:
def powerlaw_error(x, a, b, c):  # saturating power-law
    
    return a * x**(-b) + c


def relative_root_mean_squared_error(true, pred):
    num = np.sum(np.square(true - pred))
    den = np.sum(np.square(pred))
    squared_error = num/den
    rrmse_loss = np.sqrt(squared_error)
    return rrmse_loss


def root_mean_squared_error(true, pred):
    squared_error = np.square(true - pred) 
    sum_squared_error = np.sum(squared_error)
    rmse_loss = np.sqrt(sum_squared_error / len(true))
    return rmse_loss

In [ ]:
## Fit Power-law for dataset scaling Deep4

df_scaling_law_fits = pd.DataFrame(columns = ['model', 'trainset','powerlaw_error','powerlaw_error_values', 'powerlaw_error_r2_score', 'powerlaw_error_RMSE', 'powerlaw_error_RRMSE',
                                             
                                             ])


## load results 

df = pd.read_csv('./results_data_scaling_BD-Deep_trained_TUABEXB_tested_TUABEXB.csv') 

model = 'BD-Deep4'
train= 'TUABEXB'
path = "./results_Deep4Net_scaling_eval_TUABEXB_trained_TUABEXB_all_runs_scaling.csv"
df = pd.read_csv(path) 



num = len(df.groupby('size', sort=False).mean()['accuracy'])
y_values = list(100 - df.groupby('size', sort=False).mean()['accuracy'])

x_values = df.groupby('size', sort=False).mean()['accuracy'].index.astype(int).to_list()


# fit power-law based on empirical results

popt1, pcov1 = scipy.optimize.curve_fit(powerlaw_error, x_values, y_values, bounds = (0,[np.inf, np.inf, np.inf ]))

a1, b1,c1 = popt1

y_pred1 = powerlaw_error(x_values, *popt1)


#save fitted parameters 
with open('./scaling_laws_fitted_'+model+'_trained_'+train+'datascaling_popt.pkl','wb') as f: pickle.dump(popt1, f)
    
    
    
    
df_scaling_law_fits = df_scaling_law_fits.append({'model': model,  'trainset': train, 
                                                  'powerlaw_error': 'y = %.3f * (x ) **(-%.3f) + %.3f' % (a1, b1,c1), 
                                                  'powerlaw_error_values': [round(i,5) for i in popt1], 'powerlaw_error_r2_score': round(r2_score(y_values, y_pred1),4), 
                                                  'powerlaw_error_RMSE': round(root_mean_squared_error(y_values, y_pred1),4),'powerlaw_error_RRMSE': round(relative_root_mean_squared_error(y_values, y_pred1),4),

#save dataframe    
df_scaling_law_fits.to_pickle('./scaling_law_fitted_Deep.pkl')




In [ ]:
#load fitted parameters

with open('./scaling_laws_fitted_BD-Deep4_TUABEXB_datascaling_popt.pkl','rb') as f: popt1_deepexb = pickle.load(f)


# repeat loading and fitting for each model and trainingset

In [ ]:
deep_exb = pd.read_csv('./results_data_scaling_BD-Deep_trained_TUABEXB_tested_TUABEXB.csv') 
deep_comb = pd.read_csv('./results_data_scaling_BD-Deep_trained_TUABCOMB_tested_TUABEXB.csv') 

shallow_exb = pd.read_csv('./results_data_scaling_BD-Shallow_trained_TUABEXB_tested_TUABEXB.csv') 
shallow_comb = pd.read_csv('./results_data_scaling_BD-Shallow_trained_TUABCOMB_tested_TUABEXB.csv') 

tcn_exb = pd.read_csv('./results_data_scaling_BD-TCN_trained_TUABEXB_tested_TUABEXB.csv') 
tcn_comb = pd.read_csv('./results_data_scaling_BD-TCN_trained_TUABCOMB_tested_TUABEXB.csv') 


eegnet_exb = pd.read_csv('./results_data_scaling_BD-EEGNet_trained_TUABEXB_tested_TUABEXB.csv') 
eegnet_comb = pd.read_csv('./results_data_scaling_BD-EEGNet_trained_TUABCOMB_tested_TUABEXB.csv') 

In [ ]:
linewidth=3
from matplotlib.legend_handler import HandlerTuple


col_deep = '#d51965'# pink
col_shallow = '#664697' # lila
col_tcn =  '#167a72' # turquoise
col_eegnet =  '#f37651' #orange

fig = plt.figure(figsize=(24,15)) #
gs = gridspec.GridSpec(1, 2, hspace= 0.1, wspace= 0.055) 



fs=20
markersize=15
#plt.setp(axes.flat, xlabel='X-label', ylabel='Y-label')
pad = 5

ymin=13#10
ymax=41#30#25
ystep =51


splits =['BD-Deep4','BD-Shallow', 'BD-TCN', 'BD-EEGNet']#['trained on TUAB', 'trained on TUABEx']


xticklabels1 = ['100', '200', '400', '800', '1600', '3200', '6400', '7990']#list(eegnet_final_result_tuabEX.index)
posx1= [math.log2(x) for x in [100,200,400,800,1600,3200,6400,7990]]


xticklabels3 = ['100', '200', '400', '800', '1600', '3200', '6400', '7990', '10707']#list(eegnet_final_result_tuabEX.index)
posx3= [math.log2(x) for x in [100,200,400,800,1600,3200,6400,7990,10707]]

xticklabels4 = ['100', '200', '400', '800', '1600', '3200', '6400', '10707']#list(eegnet_final_result_tuabEX.index)
posx4= [math.log2(x) for x in [100,200,400,800,1600,3200,6400,10707]]


x_line = np.arange(100.00 , 10708.00)#(min(x_values), max(x_values+1), 1)



y_line_sat_deepexb = powerlaw_error(x_line, *popt1_deepexb)
y_line_sat_deepcomb = powerlaw_error(x_line, *popt1_deepcomb)

y_line_sat_shallowexb = powerlaw_error(x_line, *popt1_shallowexb)
y_line_sat_shallowcomb = powerlaw_error(x_line, *popt1_shallowcomb)

y_line_sat_tcnexb = powerlaw_error(x_line, *popt1_tcnexb)
y_line_sat_tcncomb = powerlaw_error(x_line, *popt1_tcncomb)

y_line_sat_eegnetexb = powerlaw_error(x_line, *popt1_eegnetexb)
y_line_sat_eegnetcomb = powerlaw_error(x_line, *popt1_eegnetcomb)


xticklabels5 = ['', '', '', '', '', '', '', '','']
###############
ax=plt.subplot(gs[0])#axes[0,j]
ax.set_yticks(np.linspace(100, 0, ystep, dtype=int))
#ax.set_yticklabels(yticklabels, fontsize=fs)
ax.set_ylim((ymin,ymax))
#ax.set_ylim(ypos[0],ypos[-1])
ax.yaxis.set_tick_params(labelsize=fs-2)
ylabel='Error rate [%]'
ax.set_ylabel(ylabel, fontsize=fs)
ax.tick_params(axis="y", labelsize=fs-2)
ax.set_xticks(posx3)
ax.set_xticklabels(xticklabels5, fontsize=fs, rotation=90)


ax.plot(posx1, (100 -deep_exb.groupby('size', sort=False).mean()['accuracy']),linewidth=linewidth,linestyle='none',marker='^',color=col_deep,markersize=markersize+4, fillstyle='full',markeredgewidth=2, markeredgecolor='black', markerfacecolor=col_deep ) #
ax.plot(posx1,  (100 -shallow_exb.groupby('size', sort=False).mean()['accuracy']),linewidth=linewidth,linestyle='none',marker='^',color=col_shallow, markersize=markersize+4,fillstyle='full',markeredgewidth=2, markeredgecolor='black', markerfacecolor=col_shallow) #
ax.plot(posx1, (100-tcn_exb.groupby('size', sort=False).mean()['accuracy']),linewidth=linewidth,linestyle='none',marker='^',color=col_tcn, markersize=markersize+4,fillstyle='full',markeredgewidth=2, markeredgecolor='black', markerfacecolor=col_tcn) #
ax.plot(posx1,  (100-eegnet_exb.groupby('size', sort=False).mean()['accuracy']),linewidth=linewidth,linestyle='none',marker='^',color=col_eegnet, markersize=markersize+4,fillstyle='full',markeredgewidth=2, markeredgecolor='black', markerfacecolor=col_eegnet) #


#ax.errorbar(posx1, (100 -deep_exb.groupby('size', sort=False).mean()['accuracy']),yerr=(deep_exb.groupby('size', sort=False).std()['accuracy']/np.sqrt(10)),linewidth=linewidth,linestyle='none',marker='^',color=col_deep,markersize=markersize+4, fillstyle='full',markeredgewidth=2, markeredgecolor='black', markerfacecolor=col_deep ) 

ax.set_yticks(np.linspace(100, 0, ystep, dtype=int))
#ax.set_yticklabels(yticklabels, fontsize=fs)
ax.set_ylim((ymin,ymax))
#ax.set_ylim(ypos[0],ypos[-1])
ax.yaxis.set_tick_params(labelsize=fs-2)

ylabel='Error rate [%]'
ax.set_ylabel(ylabel, fontsize=fs)
ax.tick_params(axis="y", labelsize=fs-2)
ax.set_xticks(posx3)
ax.set_xticklabels(xticklabels5, fontsize=fs, rotation=90)


ax4 = ax.twinx()


# #

ax4.set_yticks(np.linspace(100, 0, ystep, dtype=int))
# ax4.set_ylim(ypos[0], ypos[-1])
ax4.set_ylim((ymin,ymax))
ax4.get_yaxis().set_visible(False)


ax3 = ax.twinx()

ax3.get_yaxis().set_visible(False)
ax3.set_xticks(posx3)
#ax3.set_xticklabels(xticklabels3, fontsize=fs, rotation=90)
ax3.tick_params(labelleft=False) 
ax3.set_xticklabels(xticklabels5, fontsize=fs, rotation=90)
# ax3.legend(title='Train set',loc='lower left', ncol=1, fontsize=fs-5, title_fontsize=fs-5)#bbox_to_anchor=(0.775,0),


x_line = np.arange(100.00 , 10708.00)#(min(x_values), max(x_values+1), 1)



ax3.plot(posx3, np.concatenate([[np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN],y_line_sat_deepexb[[7890,-1]]],  axis=0), '--', color=col_deep,linewidth=linewidth, marker='None')
ax3.plot(posx3, np.concatenate([y_line_sat_deepexb[[0,100,300,700,1500,3100,6300,7890]], [np.NaN]], axis=0), '-', color=col_deep,linewidth=linewidth, marker='None')



ax3.plot(posx3, np.concatenate([[np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN],y_line_sat_shallowexb[[7890,-1]]],  axis=0), '--', color=col_shallow,linewidth=linewidth, marker='None')
ax3.plot(posx3, np.concatenate([y_line_sat_shallowexb[[0,100,300,700,1500,3100,6300,7890]], [np.NaN]], axis=0), '-', color=col_shallow,linewidth=linewidth, marker='None')


#
ax3.plot(posx3, np.concatenate([[np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN],y_line_sat_tcnexb[[7890,-1]]],  axis=0), '--', color=col_tcn,linewidth=linewidth, marker='None')
ax3.plot(posx3, np.concatenate([y_line_sat_tcnexb[[0,100,300,700,1500,3100,6300,7890]], [np.NaN]], axis=0), '-', color=col_tcn,linewidth=linewidth, marker='None')


ax3.plot(posx3, np.concatenate([[np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN,np.NaN],y_line_sat_eegnetexb[[7890,-1]]],  axis=0), '--', color=col_eegnet,linewidth=linewidth, marker='None')
ax3.plot(posx3, np.concatenate([y_line_sat_eegnetexb[[0,100,300,700,1500,3100,6300,7890]], [np.NaN]], axis=0), '-', color=col_eegnet,linewidth=linewidth, marker='None')


ax3.set_ylim((ymin,ymax))

ax.tick_params(axis="y", labelsize=fs-2)
ax.set_xticks(posx3)
ax.set_xticklabels(xticklabels3, fontsize=fs, rotation=90)
ax.set_xlabel('Number of training recordings\n TUABEXB', fontsize=fs)
ax.set_xticklabels(xticklabels3, fontsize=fs, rotation=90)
ax.set_yticks(np.linspace(100, 0, ystep, dtype=int))

ax.set_ylim((ymin,ymax))

p1,= ax3.plot(np.NaN, np.NaN, ls='-',linewidth=linewidth+2,
             label=' ', c=col_deep)
p2,= ax3.plot(np.NaN, np.NaN, ls='-',linewidth=linewidth+2,
             label='  ', c=col_shallow)
p3,= ax3.plot(np.NaN, np.NaN, ls='-',linewidth=linewidth+2,
             label='  ', c=col_tcn)
p4,= ax3.plot(np.NaN, np.NaN, ls='-',linewidth=linewidth+2,
             label='  ', c=col_eegnet)


t1,= ax3.plot(np.NaN, np.NaN, ls='-',linewidth=linewidth,
             label='  TUABEXB', c='black')
t2,= ax3.plot(np.NaN, np.NaN, ls='-',linewidth=linewidth,
             label='  TUABCOMB', c='black')
t3,= ax3.plot(np.NaN, np.NaN, ls='none',marker='^',fillstyle='full',markeredgewidth=2, markeredgecolor='black', markerfacecolor='black',
             label='TUABEXB', c='black',markersize=markersize-3)
t4,= ax3.plot(np.NaN, np.NaN, ls='none',marker='s',fillstyle='full',
             label='TUABCOMB', c='black',markersize=markersize-3)
ax3.get_yaxis().set_visible(False)
ax3.set_xticks(posx3)
ax3.set_xticklabels(xticklabels3, fontsize=fs, rotation=90)
ax3.tick_params(labelleft=False) 
ax3.set_xticklabels(xticklabels5, fontsize=fs, rotation=90)


ax.legend([p1,p2,p3,p4], ['BD-Deep4: $117.25n^{-0.40} + 12.78$', 'BD-Shallow: $87.65n^{-0.38} + 12.38$','BD-TCN: $100.89n^{-0.20} + 0.0$','BD-EEGNet: $167.46n^{-0.51} + 16.37$'], loc='upper right', fontsize=fs-2, ncol=1,handler_map={tuple: HandlerTuple(ndivide=None)},handlelength=2)

ax.set_xticklabels(xticklabels3, fontsize=fs, rotation=90)

###################################
ax=plt.subplot(gs[1])#axes[0,j]
ax.set_yticks(np.linspace(100, 0, ystep, dtype=int))
#ax.set_yticklabels(yticklabels, fontsize=fs)
ax.set_ylim((ymin,ymax))
#ax.set_ylim(ypos[0],ypos[-1])
ax.yaxis.set_tick_params(labelsize=fs-2)
ylabel='Error rate [%]'
#ax.set_ylabel(ylabel, fontsize=fs)
ax.tick_params(axis="y", labelsize=fs-2)
ax.set_xticks(posx3)
ax.set_xticklabels(xticklabels5, fontsize=fs, rotation=90)


ax.set_yticks(np.linspace(100, 0, ystep, dtype=int))
#ax.set_yticklabels(yticklabels, fontsize=fs)
ax.set_ylim((ymin,ymax))
#ax.set_ylim(ypos[0],ypos[-1])
ax.yaxis.set_tick_params(labelsize=fs-2)

ylabel='Error rate [%]'
#ax.set_ylabel(ylabel, fontsize=fs)
ax.tick_params(axis="y", labelsize=fs-2)
ax.set_xticks(posx3)
ax.set_xticklabels(xticklabels5, fontsize=fs, rotation=90)


ax4 = ax.twinx()

p1=ax4.plot(posx4,  (100-deep_comb.groupby('size', sort=False).mean()['accuracy']),linewidth=linewidth,linestyle='none',marker='s',color=col_deep, markersize=markersize+1, fillstyle='full',markeredgewidth=2, markeredgecolor='black', markerfacecolor=col_deep) #deep rose or rose
p2=ax4.plot(posx4,  (100-shallow_comb.groupby('size', sort=False).mean()['accuracy']),linewidth=linewidth,linestyle='none',marker='s',color=col_shallow, markersize=markersize+1,fillstyle='full',markeredgewidth=2, markeredgecolor='black', markerfacecolor=col_shallow) #
p3=ax4.plot(posx4,  (100-tcn_comb.groupby('size', sort=False).mean()['accuracy']),linewidth=linewidth,linestyle='none',marker='s',color=col_tcn, markersize=markersize+1,fillstyle='full',markeredgewidth=2, markeredgecolor='black', markerfacecolor=col_tcn) #deep rose or rose
p4=ax4.plot(posx4,  (100-eegnet_comb.groupby('size', sort=False).mean()['accuracy']),linewidth=linewidth,linestyle='none',marker='s',color=col_eegnet, markersize=markersize+1,fillstyle='full',markeredgewidth=2, markeredgecolor='black', markerfacecolor=col_eegnet) #deep rose or rose



# #

ax4.set_yticks(np.linspace(100, 0, ystep, dtype=int))
# ax4.set_ylim(ypos[0], ypos[-1])
ax4.set_ylim((ymin,ymax))
ax4.get_yaxis().set_visible(False)


ax3 = ax.twinx()

ax3.get_yaxis().set_visible(False)
ax3.set_xticks(posx3)
#ax3.set_xticklabels(xticklabels3, fontsize=fs, rotation=90)
ax3.tick_params(labelleft=False) 
ax3.set_xticklabels(xticklabels5, fontsize=fs, rotation=90)
# ax3.legend(title='Train set',loc='lower left', ncol=1, fontsize=fs-5, title_fontsize=fs-5)#bbox_to_anchor=(0.775,0),


x_line = np.arange(100.00 , 10708.00)#(min(x_values), max(x_values+1), 1)




ax3.plot(posx3, y_line_sat_deepcomb[[0,100,300,700,1500,3100,6300,7890,-1]], '-', color=col_deep,linewidth=linewidth, marker='None')

ax3.plot(posx3, y_line_sat_shallowcomb[[0,100,300,700,1500,3100,6300,7890,-1]], '-', color=col_shallow,linewidth=linewidth, marker='None')




ax3.plot(posx3, y_line_sat_tcncomb[[0,100,300,700,1500,3100,6300,7890,-1]], '-', color=col_tcn,linewidth=linewidth, marker='None')

ax3.plot(posx3, y_line_sat_eegnetcomb[[0,100,300,700,1500,3100,6300,7890,-1]], '-', color=col_eegnet,linewidth=linewidth, marker='None')



ax3.set_ylim((ymin,ymax))

ax.tick_params(axis="y", labelsize=fs-2)
ax.set_xticks(posx3)
ax.set_xticklabels(xticklabels3, fontsize=fs, rotation=90)
ax.set_xlabel('Number of training recordings\n TUABCOMB', fontsize=fs)
ax.set_xticklabels(xticklabels3, fontsize=fs, rotation=90)
ax.set_yticks(np.linspace(100, 0, ystep, dtype=int))

ax.set_ylim((ymin,ymax))

p1,= ax3.plot(np.NaN, np.NaN, ls='-',linewidth=linewidth+2,
             label=' ', c=col_deep)
p2,= ax3.plot(np.NaN, np.NaN, ls='-',linewidth=linewidth+2,
             label='  ', c=col_shallow)
p3,= ax3.plot(np.NaN, np.NaN, ls='-',linewidth=linewidth+2,
             label='  ', c=col_tcn)
p4,= ax3.plot(np.NaN, np.NaN, ls='-',linewidth=linewidth+2,
             label='  ', c=col_eegnet)


t1,= ax3.plot(np.NaN, np.NaN, ls='--',linewidth=linewidth,
             label='  TUABEXB', c='black')
t2,= ax3.plot(np.NaN, np.NaN, ls='-',linewidth=linewidth,
             label='  TUABCOMB', c='black')
t3,= ax3.plot(np.NaN, np.NaN, ls='none',marker='o',fillstyle='full',markeredgewidth=2, markeredgecolor='black', markerfacecolor='black',
             label='TUABEXB', c='black',markersize=markersize-3)
t4,= ax3.plot(np.NaN, np.NaN, ls='none',marker='s',fillstyle='full',
             label='TUABCOMB', c='black',markersize=markersize-3)
ax3.get_yaxis().set_visible(False)
ax3.set_xticks(posx3)
ax3.set_xticklabels(xticklabels3, fontsize=fs, rotation=90)
ax3.tick_params(labelleft=False) 
ax3.set_xticklabels(xticklabels5, fontsize=fs, rotation=90)


ax.legend([p1,p2,p3,p4], ['BD-Deep4: $61.80n^{-0.23} + 7.49$', 'BD-Shallow: $53.85n^{-0.30} + 11.94$','BD-TCN: $191.30n^{-0.19} + 0.0$','BD-EEGNet: $57.63n^{-0.28} + 12.78$'], loc='upper right', fontsize=fs-2, ncol=1,handler_map={tuple: HandlerTuple(ndivide=None)},handlelength=2)

ax.set_xticklabels(xticklabels3, fontsize=fs, rotation=90)

##########

plt.savefig("./scaling_law_TUABEXB-R_allmodels_1fach.pdf", bbox_inches="tight")
plt.savefig("./scaling_law_TUABEXB-R_allmodels_1fach.png", bbox_inches="tight")

plt.savefig("./scaling_law_TUABEXB-R_allmodels_1fach.jpg", bbox_inches="tight", dpi=600)



plt.show()

In [ ]:
## Model scaling

# load results 

eegnet_exb = pd.read_csv('./results_model_scaling_BD-EEGNet_trained_TUABEXB_tested_TUABEXB.csv') 

eegnet_exb['error'] = 100 - eegnet_exb['accuracy']

##....

deep_exb = pd.read_csv('./results_model_scaling_BD-Deep_trained_TUABEXB_tested_TUABEXB.csv') 
shallow_exb = pd.read_csv('./results_model_scaling_BD-Shallow_trained_TUABEXB_tested_TUABEXB.csv') 
tcn_exb = pd.read_csv('./results_model_scaling_BD-TCN_trained_TUABEXB_tested_TUABEXB.csv') 


deep_comb = pd.read_csv('./results_model_scaling_BD-Deep_trained_TUABCOMB_tested_TUABEXB.csv') 
shallow_comb = pd.read_csv('./results_model_scaling_BD-Shallow_trained_TUABCOMB_tested_TUABEXB.csv') 
tcn_comb = pd.read_csv('./results_model_scaling_BD-TCN_trained_TUABCOMB_tested_TUABEXB.csv') 
eegnet_comb = pd.read_csv('./results_model_scaling_BD-EEGNet_trained_TUABCOMB_tested_TUABEXB.csv') 

In [ ]:
### Fit and plot scaling laws model scaling


x_line_deep = np.arange(75272.00 , 1105353.00)
x_line_shallow = np.arange(9065.00 , 140643.00)
x_line_tcn = np.arange(115238.00 , 1784203.00) 
x_line_eegnet = np.arange(698.00 , 8003.00)

In [ ]:

with open('./scaling_laws_fitted_BD-EEGNet4_TUABEXB_modelsize_popt.pkl','rb') as f: popt1_eegnetexb = pickle.load(f)



In [ ]:
x_line_eegnet = np.arange(698.00 , 8003.00) #698,1202,2018,4202,8002

y_line_sat_eegnet = powerlaw_error(x_line_eegnet, *popt_eegnetexb)

In [ ]:
col_deep = '#d51965'# pink
col_shallow = '#664697' # lila
col_tcn =  '#167a72' # turquoise
col_eegnet =  '#f37651' ## orange

from matplotlib.legend_handler import HandlerTuple
fig = plt.figure(figsize=(28, 14)) #28,22
gs = gridspec.GridSpec(2,4,  wspace= 0.2, hspace=0.1) 

fs=20
markersize=18#15

pad = 5

ymin=14
ymax=21

lw=2


###############
ax=plt.subplot(gs[0])
posx= [math.log2(x) for x in [75272,143912,277052,542222,1105352]]
xticklabels = ['75,272', '143,912','277,052','542,222','1,105,352']


xticklabels = ['25', '50','100','200','400'] # rleative modelsize


ax.plot(posx, deep_exb.groupby('modelsize', sort=False).mean()['error'],linestyle='none',marker='^',color=col_deep, markersize=markersize+3, fillstyle='full',markeredgewidth=1, markeredgecolor='black', markerfacecolor=col_deep) #deep rose or rose


ax.set_yticks(np.linspace(0, 100, 101, dtype=int))
ax.set_ylim((ymin,ymax))
ax.tick_params(axis="y", labelsize=fs-2, right=False, left=True)
ax.tick_params(labelright=False, labelbottom=False) 
ylabel='Error rate [%]'
ax.set_ylabel(ylabel, fontsize=fs)

ax.set_xticks(posx)

ax.set_title('BD-Deep4', fontsize=fs)

markers = ['^']
col = ['black']#
label_set = ['TUABEXB']
lines = ['none']# '--','-']
for ss, sty in enumerate(col):
    ax.plot(np.NaN, np.NaN, ls=lines[ss],marker=markers[ss],markersize=markersize,
             label=label_set[ss], c=col[ss],linewidth=2+1)

ax.legend(title='Train set',loc='upper right', ncol=1, fontsize=fs-5, title_fontsize=fs-5, handler_map={tuple: HandlerTuple(ndivide=None)},handlelength=2)#bbox_to_anchor=(0.775,0),

ax=plt.subplot(gs[4])
posx= [math.log2(x) for x in [75272,143912,277052,542222,1105352]]
#

ax.plot(posx, deep_comb.groupby('modelsize', sort=False).mean()['error'],linestyle='none',marker='s',color=col_deep, markersize=markersize,markeredgewidth=1, markeredgecolor='black', markerfacecolor=col_deep) #deep rose or rose



ax.set_yticks(np.linspace(0, 100, 101, dtype=int))
ax.set_ylim((ymin,ymax))
ax.tick_params(axis="y", labelsize=fs-2, right=False, left=True)
ax.tick_params(labelright=False) 
ylabel='Error rate [%]'
ax.set_ylabel(ylabel, fontsize=fs)

ax.set_xticks(posx)
ax.set_xticklabels(xticklabels, fontsize=fs-2)#, rotation=45)

ax.set_xlabel('Model size [%]', fontsize=fs)


markers = ['s']
col = ['black']#col_tuab, col_tuabexb, col_tuabcomb]
label_set = ['TUABCOMB']
lines = ['none']# '--','-']
for ss, sty in enumerate(col):
    ax.plot(np.NaN, np.NaN, ls=lines[ss],marker=markers[ss],markersize=markersize,
             label=label_set[ss], c=col[ss],linewidth=2+1)

ax.legend(title='Train set',loc='upper right', ncol=1, fontsize=fs-5, title_fontsize=fs-5, handler_map={tuple: HandlerTuple(ndivide=None)},handlelength=2)#bbox_to_anchor=(0.775,0),


ax=plt.subplot(gs[1])#axes[0,j]

#ymax=21
posx= [math.log2(x) for x in [9065,18650,36722,67817,140642]]

ax.plot(posx, shallow_exb.groupby('modelsize', sort=False).mean()['error'],linestyle='none',marker='^',color=col_shallow, markersize=markersize+3,fillstyle='full',markeredgewidth=1, markeredgecolor='black', markerfacecolor=col_shallow) #deep rose or rose


ax.set_yticks(np.linspace(0, 100, 101, dtype=int))
ax.set_ylim((ymin,ymax))
ax.tick_params(axis="y", labelsize=fs-2, right=False, left=True)
ax.tick_params(labelright=False, labelbottom=False) 
ylabel='Error rate [%]'
#ax.set_ylabel(ylabel, fontsize=fs)

ax.set_xticks(posx)
ax.set_xticklabels(xticklabels, fontsize=fs-2)#, rotation=45)
ax.set_title('BD-Shallow', fontsize=fs)

ax=plt.subplot(gs[5])#axes[0,j]

#ymax=17
posx= [math.log2(x) for x in [9065,18650,36722,67817,140642]]

ax.plot(posx, shallow_comb.groupby('modelsize', sort=False).mean()['error'],linestyle='none',marker='s',color=col_shallow, markersize=markersize,markeredgewidth=1, markeredgecolor='black', markerfacecolor=col_shallow) #deep rose or rose


ax.set_yticks(np.linspace(0, 100, 101, dtype=int))
ax.set_ylim((ymin,ymax))
ax.tick_params(axis="y", labelsize=fs-2, right=False, left=True)
ax.tick_params(labelright=False) 
ylabel='Error rate [%]'
#ax.set_ylabel(ylabel, fontsize=fs)

ax.set_xticks(posx)
ax.set_xticklabels(xticklabels, fontsize=fs-2)#, rotation=45)

ax.set_xlabel('Model size [%]', fontsize=fs)


ax=plt.subplot(gs[2])#axes[0,j]



posx= [math.log2(x) for x in [115238,233846,456502,838502,1784202]]


ax.plot(posx, tcn_exb.groupby('modelsize', sort=False).mean()['error'],linestyle='none',marker='^',color=col_tcn, markersize=markersize+3,fillstyle='full',markeredgewidth=1, markeredgecolor='black', markerfacecolor=col_tcn) #deep rose or rose

ax.set_yticks(np.linspace(0, 100, 101, dtype=int))
ax.set_ylim((ymin,ymax))
ax.tick_params(axis="y", labelsize=fs-2, right=False, left=True)
ax.tick_params(labelright=False, labelbottom=False) 
ylabel='Error rate [%]'
#ax.set_ylabel(ylabel, fontsize=fs)

ax.set_xticks(posx)

ax.set_title('BD-TCN', fontsize=fs)


ax=plt.subplot(gs[6])#axes[0,j]


posx= [math.log2(x) for x in [115238,233846,456502,838502,1784202]]


ax.plot(posx, tcn_comb.groupby('modelsize', sort=False).mean()['error'],linestyle='none',marker='s',color=col_tcn, markersize=markersize,markeredgewidth=1, markeredgecolor='black', markerfacecolor=col_tcn) #deep rose or rose


ax.set_yticks(np.linspace(0, 100, 101, dtype=int))
ax.set_ylim((ymin,ymax))
ax.tick_params(axis="y", labelsize=fs-2, right=False, left=True)
ax.tick_params(labelright=False) 
ylabel='Error rate [%]'
#ax.set_ylabel(ylabel, fontsize=fs)

ax.set_xticks(posx)
ax.set_xticklabels(xticklabels, fontsize=fs-2)#, rotation=45)

ax.set_xlabel('Model size [%]', fontsize=fs)


ax=plt.subplot(gs[3])#axes[0,j]
posx= [math.log2(x) for x in [698,1202,2018,4202,8002]]

ax.plot(posx, eegnet_exb.groupby('modelsize', sort=False).mean()['error'],linestyle='none',marker='^',color=col_eegnet, markersize=markersize+3,fillstyle='full',markeredgewidth=1, markeredgecolor='black', markerfacecolor=col_eegnet) #deep rose or rose

p1,  = ax.plot(posx,y_line_sat_eegnet[[0,504,1320,3504,-1]],linestyle='-',color=col_eegnet,linewidth=lw+2)


ax.set_yticks(np.linspace(0, 100, 101, dtype=int))
ax.set_ylim((ymin,ymax))
ax.tick_params(axis="y", labelsize=fs-2, right=False, left=True)
ax.tick_params(labelright=False,labelbottom=False) 
ylabel='Error rate [%]'
#
ax.set_xticks(posx)
ax.set_xticklabels(xticklabels, fontsize=fs-2)#, rotation=45)
#
ax.set_title('BD-EEGNet', fontsize=fs)


ax.legend([p1], ['$35480.41m^{-1.45} + 17.29$'], loc='upper right', fontsize=fs-5, ncol=1,title_fontsize=fs-5,handler_map={tuple: HandlerTuple(ndivide=None)},handlelength=2)

ax=plt.subplot(gs[7])#axes[0,j]
posx= [math.log2(x) for x in [698,1202,2018,4202,8002]]
#

ax.plot(posx, eegnet_comb.groupby('modelsize', sort=False).mean()['error'],linestyle='none',marker='s',color=col_eegnet, markersize=markersize,markeredgewidth=1, markeredgecolor='black', markerfacecolor=col_eegnet) #deep rose or rose

#

p2, = ax.plot(posx,y_line_sat_eegnet_comb[[0,504,1320,3504,-1]],linestyle='-',color=col_eegnet, linewidth=lw+2)

#ax.set_yticks(np.linspace(0, 100, 101, dtype=int))
ax.set_ylim((ymin,ymax))
ax.tick_params(axis="y", labelsize=fs-2, right=False, left=True)
ax.tick_params(labelright=False) 
ylabel='Error rate [%]'
#ax.set_ylabel(ylabel, fontsize=fs)

ax.set_xticks(posx)
ax.set_xticklabels(xticklabels, fontsize=fs-2)#, rotation=45)

ax.set_xlabel('Model size [%]', fontsize=fs)


ax.legend([p2], ['$229.71m^{-0.69} + 16.06$'], loc='upper right', fontsize=fs-5, ncol=1,title_fontsize=fs-5,handler_map={tuple: HandlerTuple(ndivide=None)},handlelength=2)



plt.savefig("./scaling_modelsize_TUABEXB-R.pdf", bbox_inches="tight")
plt.savefig("./scaling_modelsize_TUABEXB-R.png", bbox_inches="tight")
plt.savefig("./scaling_modelsize_TUABEXB-R.jpg", bbox_inches="tight", dpi=600)
